In [8]:
import json
import re
nombre = 'kicillof'
filename = 'c:/Facultad/Tesis/Twitter/201910-'+nombre+'.txt'

def take_url(json_data):
    try:
        url = json_data['urls'][0]['expanded_url']
    except:
        url = ''
    return url

def replace(x):
    x = str(x)
    x = x.replace('”', '"')
    x = x.replace('“', '"')
    x = x.replace('”', '"')
    x = x.replace('\x93', '"')
    x = x.replace('\x94', '"')
    

def extract_quotes(x):
    return re.findall('"([^"]*)"', x)

def salvando_tweets(filename):
    dias = []
    frases = []
    urls = []
    urls_expected = []
    with open(filename, 'r', encoding = "utf8") as fp:
        for i, line in enumerate(fp):
            # Para cada linea lee el json y extrae la fecha
            json_data = json.loads(line)
            url = take_url(json_data)
            tweet = json_data['text']
            url = json_data['urls']
            extra = json_data



            if tweet[0:2] != 'RT':
                user = json_data['user']
                frases.append(tweet)
                dias.append(json_data['created_at'])

               # try:
               #     verified.append(user['verified'])
               # except:
               #     verified.append(False)

                try:
                    urls.append(url[0]['url'])
                    urls_expected.append(take_url(json_data), url[0]['url'])

                except:
                    urls.append(0)
                    urls_expected.append(0)

        return frases, dias, urls, urls_expected

salvando_tweets(filename)

aca http://chng.it/h4VhyM2m https://t.co/fiahWB4rwY
aca http://chng.it/sVs2qDNL https://t.co/nM6kpSTkWa
aca https://youtu.be/QpH1sg_uGrI https://t.co/FK0xcjPWQH
aca https://youtu.be/ENQ86IHcZFQ https://t.co/ctnK8HhWNV
aca https://www.eldestapeweb.com/nota/sobreseyeron-al-funcionario-de-cambiemos-que-amenazo-con-disparar-a-quienes-coloquen-carteles-de-axel-kicillof-201993019240 https://t.co/LPyrC51VSf
aca https://youtu.be/ENQ86IHcZFQ https://t.co/ctnK8HhWNV
aca https://www.eldestapeweb.com/nota/sobreseyeron-al-funcionario-de-cambiemos-que-amenazo-con-disparar-a-quienes-coloquen-carteles-de-axel-kicillof-201993019240 https://t.co/GFKDvVxCkV
aca https://www.eldisenso.com/breves/axel-kicillof-el-candidato-que-no-puede-ser-gobernador/ https://t.co/lixeXWMHyt
aca https://twitter.com/C5N/status/1178897327127445504 https://t.co/ARACdJtytK
aca http://risa.La https://t.co/j1KmmEpS1o
aca https://www.eldestapeweb.com/nota/sobreseyeron-al-funcionario-de-cambiemos-que-amenazo-con-disparar-a-quienes-

(['A LOS BONAERENSES: EXIJO UN DEBATE ENTRE MARÍA EUGENIA VIDAL Y AXEL KICILLOF - ¡Firmá la petición! https://t.co/fiahWB4rwY vía @ChangeorgAR',
  '@CardenasPozzi @alferdez @mayrasmendoza Concepto raro el tuyo de "país estable". Dólar prohibido con cepo. Dejaron el central vacío. Cerrados al mundo. Deudas de u$s 200.000 millones según Kicillof. Inflación falsa. Pobreza oculta para no estigmatizar. Corrupción que causaron muchas muertes',
  'A LOS BONAERENSES: EXIJO UN DEBATE ENTRE MARÍA EUGENIA VIDAL Y AXEL KICILLOF - ¡Firmá la petición! https://t.co/nM6kpSTkWa vía @ChangeorgAR',
  '@fabianmsm @MariaIn77662119 #apagalateleenciendetumente en todos los programas, en todos los canales, con todos los conductores se dedica 20% a Macri y 80% a Alberto Fernández sin mencionar a Cristina ni Maximo ni Kicillof',
  '@DeVincentis_J @urielnavarta @angelatorresok @realmanzur @OfeFernandez_ Ah y este videito de Kicillof que el mismo habla de la deuda de 200 mil millones de dólares \nhttps://t.co/FK0

In [39]:
import pandas as pd

def borrar_url(tweet, url):
    if urls != 0:
        try:
            t = tweet.replace(str(url), "")     
        except:
            t = tweet
    else:
        t = tweet
    return t

frases, dias, urls, verified = salvando_tweets(filename)


df = pd.DataFrame({'Tweets': frases, 'Fecha': dias, 'urls': urls, 'verified': verified})
df['Tweets_sin_url'] = df.apply(lambda row: borrar_url(row['Tweets'], row['urls']), axis = 1)
df.to_csv("Tweets_"+nombre+"_oct.csv")